In [43]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

from multiprocessing import Pool
from functools import partial

from pathlib import Path
from tqdm import tqdm

from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error

from itertools import product
# from tqdm.contrib.itertools import product

def get_stats(a, b):
    r, p = pearsonr(a, b)
    error = mae(a, b)
    rmse = mean_squared_error(a, b, squared=False)
    bias = np.mean(a - b)
    return r, error, rmse, bias


def make_stat_da(iter_num, loc_fp):
    
    param_fps = list(param_fp.glob('*'))[0]
    stems= [f.stem for f in list(param_fps.glob('*_*_*.npy'))]
    A = np.unique([s.split('_')[0] for s in stems])
    B = np.unique([s.split('_')[1] for s in stems])
    C = np.unique([s.split('_')[2] for s in stems])
    iterations = np.arange(iter_num)

    print(f'Starting {loc_fp.stem}')
    res = np.zeros((1, len(A), len(B), len(C), len(iterations)))

    da = xr.DataArray(res, coords = [[loc_fp.stem], A, B, C, iterations], dims = ['location', 'A', 'B','C', 'iteration'], name = 'pearsonr')
    res_ds = xr.merge([da, da.copy().rename('mae'), da.copy().rename('rmse'), da.copy().rename('bias')])

    lidar_orig = np.load(loc_fp.joinpath('lidar.npy'))
    elev = np.load(loc_fp.joinpath('elev.npy'))
    trees = np.load(loc_fp.joinpath('trees.npy'))

    idx = (trees < 0.5) & (elev > 2000)

    for a, b, c in product(A, B, C):
        sds_orig = np.load(loc_fp.joinpath(f'{a}_{b}_{c}.npy'))
        combo = np.vstack([lidar_orig, sds_orig])
        combo = combo.T[idx].T
        for iter in iterations:
            id_iter = np.random.choice(combo.shape[1], combo.shape[1], replace = True)
            sds, lidar = combo.T[id_iter].T
            r, mean_error, rmse, bias = get_stats(lidar, sds)
            res_ds['pearsonr'].loc[dict(location = loc_fp.stem, A = a, B = b, C = c, iteration = iter)] = r
            res_ds['mae'].loc[dict(location = loc_fp.stem, A = a, B = b, C = c, iteration = iter)] = mean_error
            res_ds['rmse'].loc[dict(location = loc_fp.stem, A = a, B = b, C = c, iteration = iter)] = rmse
            res_ds['bias'].loc[dict(location = loc_fp.stem, A = a, B = b, C = c, iteration = iter)] = bias
    
    res_ds.to_netcdf(loc_fp.parent.joinpath(loc_fp.stem + '.nc'))


out_fp = Path('/bsuhome/zacharykeskinen/scratch/spicy/param_stats.nc')

if out_fp.exists():
    print('Already exists...')
    # res_ds = xr.load_dataset(out_fp)


param_fp = Path('/bsuhome/zacharykeskinen/scratch/spicy/param_npys')

locs = list(param_fp.glob('*'))
locs = [l.stem for l in locs]

In [44]:
pool = Pool()
    
pool.map(partial(make_stat_da, 10), param_fp.glob('*'))

Starting Mores_2021-03-15
Starting Dry_Creek_2020-02-19Starting Little_Cottonwood_2021-03-18
Starting Banner_2021-03-15

Starting Mores_2020-02-09Starting Frasier_2021-03-19



KeyboardInterrupt: 